# The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [10]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [11]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content #.choices[0].message["content"]


def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [12]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [13]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

Verily, the chicken did cross the road to proclaim its audacious bravery, challenging the mundane boundaries that constrain its fowl brethren. Alas, the punchline shall remain a mystery, for a jester I am not.


In [14]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hi Isa! It's nice to meet you. How can I assist you today?


In [15]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I apologize, but as a chatbot, I don't have access to personal information. Therefore, I wouldn't know what your name is.


In [16]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa!


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [17]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [18]:
!pip install panel

     |████████████████████████████████| 20.1 MB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 700 kB/s eta 0:00:01
     |████████████████████████████████| 113 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 8.3 MB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 103 kB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 12.4 MB 8.1 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 346 kB 4.6 MB/s eta 0:00:01


In [19]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

/tmp/ipykernel_2671/3082588398.py:10: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.3, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
/home/victor/.local/lib/python3.8/site-packages/panel/viewable.py:298: ParamFutureWarning: Parameter 'object' on <class 'panel.pane.base.PaneBase'> is being given a valid parameter reference <function _param_bind.<locals>.wrapped at 0x7f4dee959dc0> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <function _param_bind.<locals>.wrapped at 0x7f4dee959dc0> will be resolved to its underlying value Column
    [0] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [1] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600). Please explicitly set allow_ref on the Parameter definition to declare whether references should be resolved or 

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [20]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

Sure! Here's a summary of your order so far:

1) Pizza:
   - Pepperoni Pizza (Size: Large) - $12.95

2) Toppings:
   - Extra Cheese - $2.00
   - Mushrooms - $1.50

3) Drinks:
   - Coke (Size: Medium) - $2.00

4) Sides:
   - Fries (Size: Regular) - $3.50

Total Price: $22.95

Is there anything else you'd like to add to your order?


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!